In [1]:
import tensorflow as tf
import _init_paths
import os
import numpy as np
from functools import partial
import tensorflow.keras.models as KM
import tensorflow.keras.backend as K
from tensorflow.keras import callbacks as KC

from utils import DisplayPlot, GetSchedular
from mydata.loader_coco import COCO
from mymodel.srgen import SrGen
from mymodel.loss import GradientPenalty, FeatLoss, MAE, WLossPos, WLossNeg
import cfg

In [2]:
import argparse
def GetTrainingOptions(devices = '2', # gpu devices to use
                       batch = 12,
                       ratio = 2, # resolution ratio between low and high resolution images
                       lr = 2e-4, # learning rate
                       epochs = 20, # epochs to train
                       size_train = 800, # num of training data per epoch
                       size_val = 100, # num of validation data per epoch
                       dir_model = 'save_model', # root directory for saving results
                       ):
    
    opt = argparse.Namespace()
    opt.devices = devices
    opt.batch = batch
    opt.ratio = ratio
    opt.lr = lr
    opt.epochs = epochs
    opt.size_train = size_train
    opt.size_val = size_val
    opt.dir_model = dir_model
    return opt


In [3]:
opt = GetTrainingOptions()

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = opt.devices

### dataset

In [5]:
ds = COCO(root=cfg.PATH_COCO, ratio=opt.ratio, batch_size=opt.batch)
ds_tr, ds_val = ds.GetDataset()

### Callback

In [6]:


display = DisplayPlot(root='Imgs', ds=ds)

if not os.path.isdir(opt.dir_model): os.makedirs(opt.dir_model)
ckpt = KC.ModelCheckpoint(monitor='val_loss', 
                          filepath=os.path.join(opt.dir_model, 'weights.h5'), 
                          save_weights_only=True, 
                          save_best_only=True, mode='min')

Schedular = GetSchedular(lr_base=opt.lr, epoch_total=opt.epochs)
reduce_lr = KC.LearningRateScheduler(Schedular)

callbacks = [ckpt, display, reduce_lr]

### Train

In [7]:
model = SrGen(input_shape=ds.GetShapeLow())
optimizer = tf.keras.optimizers.Adam(opt.lr)
model.compile(optimizer=optimizer, loss='mae')

In [8]:
model.fit(ds_tr,
          steps_per_epoch = opt.size_train,
          epochs = opt.epochs, 
          verbose = 1,
          validation_data = ds_val,
          validation_steps = opt.size_val,
          callbacks = callbacks)

Epoch 1/20
800/800 [==============================] - 149s 186ms/step - loss: 0.0954 - val_loss: 0.0910 - lr: 2.0000e-04
Epoch 2/20
800/800 [==============================] - 144s 180ms/step - loss: 0.0708 - val_loss: 0.0630 - lr: 1.9098e-04
Epoch 3/20
800/800 [==============================] - 145s 181ms/step - loss: 0.0662 - val_loss: 0.1053 - lr: 1.8191e-04
Epoch 4/20
800/800 [==============================] - 144s 181ms/step - loss: 0.0654 - val_loss: 0.0759 - lr: 1.7279e-04
Epoch 5/20
800/800 [==============================] - 146s 182ms/step - loss: 0.0632 - val_loss: 0.0609 - lr: 1.6361e-04
Epoch 6/20
800/800 [==============================] - 146s 182ms/step - loss: 0.0602 - val_loss: 0.0594 - lr: 1.5438e-04
Epoch 7/20
800/800 [==============================] - 146s 182ms/step - loss: 0.0586 - val_loss: 0.0635 - lr: 1.4508e-04
Epoch 8/20
800/800 [==============================] - 145s 181ms/step - loss: 0.0571 - val_loss: 0.0550 - lr: 1.3572e-04
Epoch 9/20
800/800 [============